<h1 align="center"><font color="yellow">PGVector: Crie, armazene e consulte Embeddings OpenAI no PostgreSQL usando pgvector</font></h1>

<font color="yellow">Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro</font>

Vamos seguir os seguintes passos:

* docker pull ankane/pgvector

* docker-compose up -d

* Connect to the server 





In [7]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.pgvector import PGVector, DistanceStrategy
from langchain.docstore.document import Document

import os
import openai
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = os.getenv('OPENAI_API_KEY')


embeddings = OpenAIEmbeddings()

connection_string = PGVector.connection_string_from_db_params(
    driver=os.environ.get("DB_DRIVER", "psycopg"),
    host=os.getenv('DB_HOST'),
    port=os.getenv('DB_PORT'),
    database=os.getenv('DB_NAME'),
    user=os.getenv('DB_USER'),
    password=os.getenv('DB_PASSWORD')
)


In [8]:
loader = TextLoader('./data/exemplo_SEEDF.txt')
documents = loader.load()

In [9]:
documents

[Document(page_content='Reforço nas classes hospitalares\nInscrições iniciam dia 7 de fevereiro. São 15 vagas para atuar em dez hospitais\nCom novos hospitais incluídos, o processo seletivo de professor da educação básica- carreira magistério, \npara atuar nas classes hospitalares foi publicado nesta segunda-feira (31), no Diário Oficial do DF. De cinco, \nagora somam 11 as unidades que contam com o programa. Esta portaria da Secretaria de Educação em conjunto com \na Secretaria de Saúde garante atendimento educacional às crianças da educação infantil e dos anos iniciais do \nensino fundamental, matriculadas na rede pública de ensino, impossibilitadas de frequentar as escolas em razão \nde tratamento médico. Serão três etapas: inscrição, análise curricular documental e entrevista individual, \nconforme critérios definidos no edital.\n\nRequisitos necessários\nEstão aptos a participar professores de educação básica da carreira magistério do DF com \ncarga horária de 40 horas semanais e 

In [10]:
text_splitter = CharacterTextSplitter(chunk_size=300,
                                      chunk_overlap=50,
                                      separator="\n"
                                     )

docs = text_splitter.split_documents(documents)
print(len(documents))
print(len(docs))

1
10


In [11]:
docs

[Document(page_content='Reforço nas classes hospitalares\nInscrições iniciam dia 7 de fevereiro. São 15 vagas para atuar em dez hospitais\nCom novos hospitais incluídos, o processo seletivo de professor da educação básica- carreira magistério,', metadata={'source': './data/exemplo_SEEDF.txt'}),
 Document(page_content='para atuar nas classes hospitalares foi publicado nesta segunda-feira (31), no Diário Oficial do DF. De cinco, \nagora somam 11 as unidades que contam com o programa. Esta portaria da Secretaria de Educação em conjunto com', metadata={'source': './data/exemplo_SEEDF.txt'}),
 Document(page_content='a Secretaria de Saúde garante atendimento educacional às crianças da educação infantil e dos anos iniciais do \nensino fundamental, matriculadas na rede pública de ensino, impossibilitadas de frequentar as escolas em razão', metadata={'source': './data/exemplo_SEEDF.txt'}),
 Document(page_content='de tratamento médico. Serão três etapas: inscrição, análise curricular documental 

In [12]:
from typing import List, Tuple

collection_name = 'SEEDF'
# O Módulo PGVector tentará criar uma tabela com o nome da coleção (collection). 
# Portanto, certifique-se de que o nome da coleção seja exclusivo e que o usuário tenha permissão para criar uma tabela.
db = PGVector.from_documents(
    embedding=embeddings,
    documents=docs,
    collection_name=collection_name,
    connection_string=connection_string,
)


In [13]:
query ="Quantos locais de Classes Hospitalares no DF há?" #"Quais são os requisitos necessários para a educação hospitalar?"
docs_with_score: List[Tuple[Document, float]] = db.similarity_search_with_score(query,
                                                                                k=2)


In [14]:
docs_with_score

[(Document(page_content='Locais de atuação\nSão 15 vagas e cadastro reserva para atuar nas classes hospitalares:\n▸ Hospital Materno Infantil (HMIB)\n▸ Hospital Regional da Asa Norte (HRAN)\n▸ Hospital Regional de Taguatinga (HRT)\n▸ Hospital Regional de Ceilândia (HRC)\n▸ Hospital Regional do Gama (HRG)', metadata={'source': './data/exemplo_SEEDF.txt'}),
  0.10183656915212536),
 (Document(page_content='para atuar nas classes hospitalares foi publicado nesta segunda-feira (31), no Diário Oficial do DF. De cinco, \nagora somam 11 as unidades que contam com o programa. Esta portaria da Secretaria de Educação em conjunto com', metadata={'source': './data/exemplo_SEEDF.txt'}),
  0.11261681384071764)]

In [ ]:
for doc, score in docs_with_score:
    print("-" * 100)
    print("Score: ", score)
    print("Conteúdo: ", doc.page_content)
    print("")
    print(doc.metadata)
    print("-" * 100)
    

In [ ]:
store = PGVector(
    connection_string=connection_string, 
    embedding_function=embeddings, 
    collection_name= 'SEEDF', #'Eddy_RESUMINHO_estado_da_Uniao',
    distance_strategy=DistanceStrategy.COSINE
)

retriever = store.as_retriever(search_kwargs={"k": 3})


In [ ]:
retriever

In [ ]:

retriever.get_relevant_documents(query="Quantos locais de Classes Hospitalares no DF há?")
